In [2]:
import json
from pprint import pprint

import numpy as np

import collections
import operator

In [36]:
answers = json.load(open('Annotations/mscoco_val2014_annotations.json'))

## Baseline 2 (always "yes")

In [56]:
c = 0.0
cat_counter = {'yes/no': 0, 'other': 0, 'number': 0, 'all': 0}
cat = {'yes/no': 0, 'other': 0, 'number': 0, 'all': 0}
for obj in answers['annotations']:

    cat_counter[obj['answer_type']] += 1
    
    l = []
    for a in obj['answers']:
        l.append(a['answer'])
    
    count = l.count('yes')
    if count > 2:
        cat[obj['answer_type']] += 1
        c += 1
    else:
        cat[obj['answer_type']] += float(count)/3
        c += float(count)/3

print('other accuracy on validation set: %.02f' % (100 * cat['other']/cat_counter['other']))
print('number accuracy on validation set: %.02f' % (100 * cat['number']/cat_counter['number']))
print('yes/no accuracy on validation set: %.02f' % (100 * cat['yes/no']/cat_counter['yes/no']))
print('final accuracy on validation set: %.02f' % (100 * c/len(answers['annotations'])))

other accuracy on validation set: 1.53
number accuracy on validation set: 0.42
yes/no accuracy on validation set: 72.00
final accuracy on validation set: 27.76


## Baseline 3 (q-Prior)

In [79]:
questions_data = ['Questions/OpenEnded_mscoco_train2014_questions.json']
                  #'Questions/OpenEnded_mscoco_val2014_questions.json']
                  #'Questions/OpenEnded_abstract_v002_train2015_questions.json', 
                  #'Questions/OpenEnded_abstract_v002_val2015_questions.json']

In [80]:
annotations_data = ['Annotations/mscoco_train2014_annotations.json']
                    #'Annotations/mscoco_val2014_annotations.json']
                    #'Annotations/abstract_v002_train2015_annotations.json',
                    #'Annotations/abstract_v002_val2015_annotations.json']

In [87]:
def createDictBaseline3(*args):
    qCat = collections.defaultdict(dict)
    
    for dataset in list(*args):
        data = json.load(open(dataset))
        
        for i in range(len(data["annotations"])):
            question_type = data["annotations"][i]["question_type"]
            answer = data["annotations"][i]['multiple_choice_answer']
            qCat[question_type][answer] = 1 if answer not in qCat[question_type] else qCat[question_type][answer] + 1
    
    #print(qCat)
    
    qPrior = dict()
    for k, v in qCat.items():
        qPrior[k] = sorted(qCat[k].items(), key=operator.itemgetter(1), reverse=True)[0][0]
    
    return qPrior

In [88]:
# return a dictionary such that:
# dict[question_type] = most_popular_answer_by_question_type
qPrior = createDictBaseline3(annotations_data)

In [89]:
qPrior

{'are': 'no',
 'are the': 'yes',
 'are there': 'yes',
 'are there any': 'yes',
 'are these': 'yes',
 'are they': 'yes',
 'can you': 'yes',
 'could': 'yes',
 'do': 'yes',
 'do you': 'yes',
 'does the': 'yes',
 'does this': 'yes',
 'has': 'yes',
 'how': 'very',
 'how many': '2',
 'how many people are': '2',
 'how many people are in': '2',
 'is': 'yes',
 'is he': 'yes',
 'is it': 'yes',
 'is that a': 'yes',
 'is the': 'yes',
 'is the man': 'yes',
 'is the person': 'yes',
 'is the woman': 'yes',
 'is there': 'yes',
 'is there a': 'yes',
 'is this': 'yes',
 'is this a': 'yes',
 'is this an': 'yes',
 'is this person': 'yes',
 'none of the above': 'yes',
 'was': 'yes',
 'what': 'baseball',
 'what animal is': 'cat',
 'what are': 'frisbee',
 'what are the': 'grass',
 'what brand': 'apple',
 'what color': 'red',
 'what color are the': 'white',
 'what color is': 'red',
 'what color is the': 'white',
 'what does the': 'stop',
 'what is': 'snow',
 'what is in the': 'kite',
 'what is on the': 'snow'

In [90]:
c = 0.0
cat_counter = {'yes/no': 0, 'other': 0, 'number': 0, 'all': 0}
cat = {'yes/no': 0, 'other': 0, 'number': 0, 'all': 0}
for obj in answers['annotations']:

    cat_counter[obj['answer_type']] += 1
    
    l = []
    for a in obj['answers']:
        l.append(a['answer'])
    
    count = l.count(qPrior[obj['question_type']])
    if count > 2:
        cat[obj['answer_type']] += 1
        c += 1
    else:
        cat[obj['answer_type']] += float(count)/3
        c += float(count)/3

print('other accuracy on validation set: %.02f' % (100 * cat['other']/cat_counter['other']))
print('number accuracy on validation set: %.02f' % (100 * cat['number']/cat_counter['number']))
print('yes/no accuracy on validation set: %.02f' % (100 * cat['yes/no']/cat_counter['yes/no']))
print('final accuracy on validation set: %.02f' % (100 * c/len(answers['annotations'])))

other accuracy on validation set: 9.08
number accuracy on validation set: 32.61
yes/no accuracy on validation set: 72.13
final accuracy on validation set: 35.63


## Other baselines

# Baseline 1: Random

In [91]:
def top_answers_vqa_training(*args):
    qCat = collections.defaultdict(dict)
    
    for dataset in list(*args):
        data = json.load(open(dataset))
        
        for i in range(len(data["annotations"])):
            answer = data["annotations"][i]['multiple_choice_answer']
            qCat[answer] = 1 if answer not in qCat else qCat[answer] + 1
    
    sorted_answer = sorted(qCat.items(), key=operator.itemgetter(1), reverse=True)
    top_answer=sorted_answer[0:1000]
    
    return top_answer

In [92]:
top_answer = top_answers_vqa_training(annotations_data)

In [105]:
import random

c = 0.0
cat_counter = {'yes/no': 0, 'other': 0, 'number': 0, 'all': 0}
cat = {'yes/no': 0, 'other': 0, 'number': 0, 'all': 0}
for obj in answers['annotations']:

    cat_counter[obj['answer_type']] += 1
    
    l = []
    for a in obj['answers']:
        l.append(a['answer'])
    
    count = l.count(random.choice(top_answer)[0])
    if count > 2:
        cat[obj['answer_type']] += 1
        c += 1
    else:
        cat[obj['answer_type']] += float(count)/3
        c += float(count)/3

print('other accuracy on validation set: %.02f' % (100 * cat['other']/cat_counter['other']))
print('number accuracy on validation set: %.02f' % (100 * cat['number']/cat_counter['number']))
print('yes/no accuracy on validation set: %.02f' % (100 * cat['yes/no']/cat_counter['yes/no']))
print('final accuracy on validation set: %.02f' % (100 * c/len(answers['annotations'])))

other accuracy on validation set: 0.12
number accuracy on validation set: 0.14
yes/no accuracy on validation set: 0.12
final accuracy on validation set: 0.12


# Baseline 4: K-nearest

In [5]:
import skipthoughts

In [6]:
model = skipthoughts.load_model()

Loading model parameters...
Compiling encoders...
Loading tables...
Packing up...


In [7]:
encoder = skipthoughts.Encoder(model)

In [ ]:
#import nltk
#nltk.download()
X=['What is your name','are you there']
vectors = encoder.encode(X)
vectors

In [ ]:
np.shape(vectors)

In [ ]:
def question_list(*args):
    #Function which compute the list of questions,the list of question_id and the list of image_id 
    #qCat = collections.defaultdict(dict)
    question_vec=[]
    question_id_vec=[]
    vector_image_id=[]
    #for dataset in list(*args):
    data = json.load(open(*args))
    for i in range(len(data["questions"])):
        #question_type = data["annotations"][i]["question_type"]
        question = data["questions"][i]['question']
        image_id=data["questions"][i]['image_id']
        question_id=data["questions"][i]['image_id']
        vector_image_id.append(image_id)
        question_vec.append(question)
        question_id_vec.append(question_id)
    return (question_vec,vector_image_id,question_id_vec)

In [ ]:
[question_vec,vector_image_id,question_id_vec]=question_list(questions_data)
vector=encoder.encode(question_vec)
np.savetxt('test.out', vector, delimiter=',')

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
25


In [ ]:
np.shape(vector)

In [ ]:
def answer_list(*args):
    #Function that list the answer vec, the image id vec and the question id vec 
    #qCat = collections.defaultdict(dict)
    answer_vec=[]
    question_id_vec=[]
    vector_image_id=[]
    data = json.load(open(*args))
    for i in range(len(data["annotations"])):
        answer = data["annotations"][i]['multiple_choice_answer']
        image_id=data["annotations"][i]['image_id']
        question_id_vec=data["annotations"][i]['question_id']
        answer_vec.append(answer)
        vector_image_id.append(image_id)
    return (answer_vec,vector_image_id,question_id_vec)

In [ ]:
[answer_vec,vector_image_id_ans,question_id_vec_ans]=answer_list(annotations_data)

In [ ]:
np.shape(vector)

In [ ]:
question

In [ ]:
import sys
sys.path.append('/Users/tiomokomalik/caffe-master/python')

# and then import caffe
import caffe
import pickle
import numpy as np

In [ ]:
import os
import os.path
def extract_features(image_id,n,test):
    #image_number=np.shape(image_id);
    iter=np.arange(n)
    features_vec=[];
    for i in iter:
        if n==1:
            image_id_one=image_id
        else :
            image_id_one=image_id[i]
        #compute the number od zeros to add
        count=12-len(str(image_id_one));
        vector=np.tile(0, count)
        vector1=np.append(vector,image_id_one)
        number_image="".join(str(x) for x in vector1)
        if test==0:
            file_path1='/Volumes/TIOMOKO/train2014/COCO_train2014_'+number_image+'.jpg'
            file_path2='/Volumes/TIOMOKO/val2014/COCO_val2014_'+number_image+'.jpg'
            input_images_file1="".join(file_path1)
            input_images_file2="".join(file_path2)
            if os.path.isfile(input_images_file1)==True :
                input=file_path
            else :
                input=file_path2
        else :
            input='/Volumes/TIOMOKO/test2015/COCO_test2015_'+number_image+'.jpg'
        input_images_file="".join(input)
        #Function which compute the fc7 features (output) of an image (input) from Caffe model.
        #This function has been written and inspired strongly on the webpages
        #https://prateekvjoshi.com/2016/04/26/how-to-extract-feature-vectors-from-deep-neural-networks-in-python-caffe/

        #Deploy.prototxt copied in model_def
        model_def= "/Users/tiomokomalik/caffe-master/models/bvlc_reference_caffenet/deploy.prototxt"
        #Load the pretrained model
        pretrained_model="/Users/tiomokomalik/Dropbox/Object_reco/Code/bvlc_reference_caffenet.caffemodel"

        # output file to write extracted features to disk
        #output_file = "/Users/tiomokomalik/Dropbox/Object_reco/Code/output-file.pkl"

        #Initialize the convolutional neural network
        net = caffe.Net(model_def, pretrained_model, caffe.TEST)
        #Features to extract and insure it's in the model
        layer = "fc7"
        if layer not in net.blobs:
            raise TypeError("Invalid layer name: " + layer)

        #Specify the imagemean file for the image transformer
        imagemean_file = '/Users/tiomokomalik/caffe-master/python/caffe/imagenet/ilsvrc_2012_mean.npy'
        #Transform the image before feeding up to the network
        transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
        transformer.set_mean('data', np.load(imagemean_file).mean(1).mean(1))
        transformer.set_transpose('data', (2,0,1))
        transformer.set_raw_scale('data', 255.0)
        #Reshape the network blob
        net.blobs['data'].reshape(1,3,227,227)
        #Load the input image:
        img = caffe.io.load_image(input_images_file)
        #Run the image through the preprocessor:
        net.blobs['data'].data[...] = transformer.preprocess('data', img)
        #Run the image through the network:
        output = net.forward()
        #Extract the output
        vector=net.blobs[layer].data[0]
        features_vec.append(vector)
    return features_vec

In [ ]:
image_id=[458748,9]
features=extract_features(image_id)

In [ ]:
np.shape(features)

In [ ]:
number_image

In [ ]:
for dataset in list(annotations_data):
        data = json.load(open(dataset))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
baseline4 = []
k=4;
for q in yes_test_OE['questions']:
    #First find the k closest question in the list
    #For that define the cosine similarity
    query=q['question']
    query_vec=[]
    query_vec.append(query)
    vector_query=encoder.encode(query_vec)
    cosine_distance=cosine_similarity(vector_query,vector)
    length=np.shape(cosine_distance)[1]-1
    #Sort the cosine similarity and conserve the k first index
    index_sorted=np.argsort(cosine_distance)
    index_sorted_select=index_sorted[0][0:k]
    #From the question sorted find the corresponding image
    image_id_query=q['image_id']
    image_id=[vector_image_id[i] for i in index_sorted_select]
    #Compute the cosine similarity between the image query and the k image_id
    #First compute the fc7 feature for each image
    fc7_image_id_query=extract_features(image_id_query,1,1)
    fc7_image_id=extract_features(image_id,4,0)
    cosine_distance_image=cosine_similarity(fc7_image_id_query,fc7_image_id)
    index_sorted_image=np.argsort(cosine_distance_image)
    index_sorted_select_image=index_sorted_image[0][0]
    index_best_answer=vector_image_id_ans.index(image_id[index_sorted_select_image])
    #chhosen_image_id=image_id(index_sorted_image[0])
    #find the resulting answer corresponding to this image id
    answer_image_id=answer_vec[index_best_answer]
    d['answer'] = answer_image_id
    d['question_id'] = q['question_id']
    baseline4.append(d)
    print 'done'

with open('baseline4.json', 'w') as outfile:
    json.dump(baseline4, outfile)

In [ ]:
np.arange(k)

In [ ]:
for q in yes_test_OE['questions']:
    #First find the k closest question in the list
    #For that define the cosine similarity
    query=q['question'];

In [ ]:
query_vec=[];
query_vec.append(query)

In [ ]:
encoder.encode(query_vec)